# Example for using fleet modeling in AeroMAPS

This notebook aims at using the classes dedicated to the fleet modelling that are used in AeroMAPS.

## Link with other AeroMAPS models

This fleet model can be used in AeroMAPS for improving the modeling of the fleet efficiency. For this purpose, a dedicated model is loaded, based on the bottom-up approach using fleet model. It replaces the basic top-down approach for modeling the evolution of the fleet efficiency, based on annual efficiency gains.

In [ ]:
%matplotlib widget
from pathlib import Path

from aeromaps import create_process

### Use the tutorial-specific configuration
This tutorial ships with `data/config.yaml`, which keeps the fleet YAML, aircraft inventory, and output files next to the notebook. Update that YAML (and the referenced YAML files) to explore alternative fleets.

In [ ]:
CONFIG_PATH = Path("data") / "config.yaml"

In [ ]:
process = create_process(configuration_file=str(CONFIG_PATH))

Here, an example using the previous fleet generated is performed with the AeroMAPS default float inputs. The hydrogen used for hydrogen aircraft is assumed to be produced via electrolysis using dedicated low-carbon electricity.

In [ ]:
# Change end year
process.parameters.end_year = 2060
process.common_setup()
process.setup_mda()

In [ ]:
# Hydrogen production
process.parameters.hydrogen_electrolysis_share_reference_years = []
process.parameters.hydrogen_electrolysis_share_reference_years_values = [100]
process.parameters.hydrogen_gas_ccs_share_reference_years = []
process.parameters.hydrogen_gas_ccs_share_reference_years_values = [0]
process.parameters.hydrogen_coal_ccs_share_reference_years = []
process.parameters.hydrogen_coal_ccs_share_reference_years_values = [0]
process.parameters.hydrogen_gas_share_reference_years = []
process.parameters.hydrogen_gas_share_reference_years_values = [0]
# Electricity emission factor
process.parameters.electricity_emission_factor_reference_years = []
process.parameters.electricity_emission_factor_reference_years_values = [20.0]

## Load and process

In [ ]:
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

The fleet is divided into three main categories of aircraft, associated to an initial subcategory based on representative aircraft:
- Short range (Conventional narrow-body)
- Medium range (Conventional narrow-body)
- Long range (Conventional wide-body)

Lets now assemble these categories into a fleet object. By default, two new aircraft have been added to the initial subcategories, and two subcategories have also been added for Short Range (turboprop aircraft and hydrogen aircraft). Illustrative values are considered.

In [ ]:
fleet = process.fleet
fleet_model = process.fleet_model

## Set up the fleet

### Inspect the configured fleet
`Fleet.pretty_print()` now highlights each category, its calibration references, and every additional aircraft defined in the YAML files. Pass `absolute=True` (optionally `reference="old"`) to convert the aircraft deltas into absolute values using the chosen reference aircraft as a baseline.

In [ ]:
# Relative deltas (default display)
fleet.pretty_print()

print("\nAbsolute values vs recent reference:")
fleet.pretty_print(absolute=True)

A dedicated configuration is now provided through the YAML files `fleet/config/aircraft_catalog.yaml` and `fleet/config/fleet.yaml`. Update those files (or point `Fleet` to custom copies) to change the future composition of the fleet: add subcategories, set their shares, and list the aircraft IDs assigned to each one. The helper cell below shows how to inspect the currently loaded configuration inside this notebook.

In [ ]:
from pprint import pprint
from aeromaps.utils.yaml import read_yaml_file

fleet_config_path = process.fleet.fleet_config_path
print(f"Fleet configuration loaded from: {fleet_config_path}")
fleet_config = read_yaml_file(str(fleet_config_path))
short_range_cfg = next(cat for cat in fleet_config["categories"] if cat["id"] == "short_range")
pprint(short_range_cfg)

You can change fleet renewal rates for each category. The default value is 25 years.

In [ ]:
fleet.categories["Medium Range"].parameters.life = 20

## Compute and results for the fleet model

Once the fleet has been chosen, the user can compute.

In [ ]:
fleet_model.compute()

The user can then display the results.

In [ ]:
fleet_model.df

More simply, the user can plot fleet renewal and its impact on energy consumption per ASK for each category.

In [ ]:
fleet_model.plot()

## Compute and results for the global model

The user can finally run the global model for studying a prospective scenario for air transport based on a hand-made fleet.

In [ ]:
process.compute()
process.write_json()

In [ ]:
process.plot("air_transport_co2_emissions")

In [ ]:
# Verify the outputs between .outputs.json and data/reference/outputs.json
from aeromaps.utils.functions import compare_json_files

files_are_different = compare_json_files(
    "./data/reference/outputs.json",
    "./data/outputs.json",
    rtol=0.0001,
    atol=0,
)

if files_are_different:
    raise ValueError("The outputs.json files are different.")

In [ ]:
from aeromaps.utils.functions import clean_notebooks_on_tests

clean_notebooks_on_tests(globals())